# MERRA2 SLV+Radiation Monthly Granularity Test
- Download
- Trim
- Create Cutout
- Generate Solar Outputs

## 1. Import and Download Data

In [ ]:
import calendar
import geodata
import xarray as xr
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)
DS_monthly = geodata.Dataset(module="merra2",
					 years=slice(1997, 1997),
					 months=slice(1,1),
                     weather_data_config = "slv_radiation_monthly")  

In [ ]:
if DS_monthly.prepared == False:
	DS_monthly.get_data()

## 2. Trim Variables

In [ ]:
## Variables before trimming
ds = xr.open_dataset(DS_monthly.downloadedFiles[0][1])
ds.data_vars

In [ ]:
DS_monthly.trim_variables()

In [ ]:
## Variables after trimming
ds = xr.open_dataset(DS_monthly.downloadedFiles[0][1])
ds.data_vars

## 3. Create Cutout

In [ ]:
monthly_cutout = geodata.Cutout(name="monthly-test-cutout",
                       module="merra2",
                       weather_data_config="slv_radiation_monthly",
                       xs=slice(138.5, 139.5),
                       ys=slice(35, 36),
                       years=slice(1997, 1997),
                       months=slice(1,1))

In [ ]:
monthly_cutout.prepare()

In [ ]:
# Verify Cutout Contents
monthly_cutout.__dict__

## 4. Generate Solar Output

In [ ]:
ds_pv = geodata.convert.pv(
    monthly_cutout, 
    panel="KANENA", 
    orientation = "latitude_optimal"
    )
ds_pv

In [ ]:
df_pv = ds_pv.to_dataframe(name='pv')
df_pv

In [ ]:
df_pv.to_csv('pv_test_monthly.csv')